<a href="https://colab.research.google.com/github/MichaelHuang05/test-repo/blob/master/FintechDR_PCA_CNN_with_replacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from google.colab import drive
drive.mount('/content/Drive')


path = "/content/Drive/My Drive/Colab Notebooks/images"
os.chdir(path) 
with open('/content/Drive/My Drive/Colab Notebooks/labels.json') as label:
    labels = json.load(label)

label_df = pd.DataFrame(labels, columns = ["file_name", "DR_stage"])
label_df["Eye_threaten"] = 0
label_df["Eye_threaten"][label_df["DR_stage"] > 1] = 1


Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
#### read images and flat it

filenames = label_df["file_name"]
feature = []

for i in range(len(filenames)):              
    temp = cv2.imread(path + "/" + filenames[i])
    temp = cv2.resize(temp, (512,512))
    feature.extend(temp)

feature_mat = np.matrix(np.reshape(feature, (1500,512*512*3)))
feature_df = pd.DataFrame(feature_mat)

In [3]:
#### combining label and feature

fullset = pd.merge(label_df, feature_df, left_index=True, right_index=True)
fullset.head()

,file_name,DR_stage,Eye_threaten,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,786392,786393,786394,786395,786396,786397,786398,786399,786400,786401,786402,786403,786404,786405,786406,786407,786408,786409,786410,786411,786412,786413,786414,786415,786416,786417,786418,786419,786420,786421,786422,786423,786424,786425,786426,786427,786428,786429,786430,786431
0,28466_left.jpeg,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,...,6,8,9,7,7,8,6,6,7,5,6,7,5,6,7,5,6,7,5,7,8,6,7,8,6,7,8,6,7,8,6,6,7,5,6,7,5,6,7,5
1,38420_left.jpeg,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,42796_right.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
3,7686_right.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,30206_right.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1


In [4]:
#### training and testing set split

from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
import random


random.seed( 30 )

y_train, y_test = train_test_split(fullset.iloc[:,0:3], test_size=0.2)



####################################

q = pd.DataFrame(y_train)
a = q[q["Eye_threaten"].isin([0])]
b = q[q["Eye_threaten"].isin([1])]


random.seed( 30 )
boota = np.random.choice(a["file_name"], replace = True, size = 600)
bootb = np.random.choice(b["file_name"], replace = True, size = 600)


### later use boot_df
boota_df = pd.DataFrame(boota, columns = ["file_name"])
bootb_df = pd.DataFrame(bootb, columns = ["file_name"])


train_filenames_df = pd.concat([boota_df, bootb_df])
y_new_train = pd.merge(train_filenames_df, label_df.iloc[:,0:3], how = "left", on = ["file_name", "file_name"])
x_new_train = pd.merge(train_filenames_df, fullset, how = "left", on = ["file_name", "file_name"])
x_test = pd.merge(y_test.iloc[:,0], fullset, how = "left", on = ["file_name", "file_name"])


del x_new_train['Eye_threaten']
del x_new_train['file_name']
del x_new_train['DR_stage']
del x_test['Eye_threaten']
del x_test['file_name']
del x_test['DR_stage']
del fullset
#################################### 



# Onehot encoding for catogory
y_new_trainOneHot = to_categorical(y_new_train.iloc[:,2])
y_testOneHot = to_categorical(y_test.iloc[:,2])

Using TensorFlow backend.


In [5]:
#### PCA to do dimension reduction

from sklearn.decomposition import PCA
import time

n = 100
t0 = time.time()
pca = PCA(n_components=n, whiten=True).fit(x_new_train)
print("pca done %0.3fs" % (time.time() - t0))

print(sum(pca.explained_variance_ratio_))

t0 = time.time()
x_train_pca = pca.transform(x_new_train)
x_test_pca  = pca.transform(x_test)
print("data set to pca done %0.3fs" % (time.time() - t0))

pca done 124.333s
0.9855693206395447
data set to pca done 9.219s


In [69]:
### NN 
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()


model.add(Dense(64, input_dim = n, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))
    
model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))
    
model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))
    
model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(64, activation = "relu"))

model.add(Dropout(0.2))







model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
         

# maybe should change to focal loss?


model.summary()


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_157 (Dense)            (None, 64)                6464      
_________________________________________________________________
dropout_145 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_158 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_146 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_159 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_147 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_160 (Dense)            (None, 64)              

In [70]:
model.fit(x = x_train_pca, y = y_new_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Train on 1080 samples, validate on 120 samples
Epoch 1/1000
 - 4s - loss: 0.7164 - acc: 0.4981 - val_loss: 0.7737 - val_acc: 0.0000e+00
Epoch 2/1000
 - 0s - loss: 0.6883 - acc: 0.5454 - val_loss: 0.7757 - val_acc: 0.0000e+00
Epoch 3/1000
 - 0s - loss: 0.6898 - acc: 0.5352 - val_loss: 0.7714 - val_acc: 0.0167
Epoch 4/1000
 - 0s - loss: 0.6843 - acc: 0.5593 - val_loss: 0.7815 - val_acc: 0.0000e+00
Epoch 5/1000
 - 0s - loss: 0.6836 - acc: 0.5630 - val_loss: 0.7795 - val_acc: 0.0083
Epoch 6/1000
 - 0s - loss: 0.6678 - acc: 0.5713 - val_loss: 0.7972 - val_acc: 0.0583
Epoch 7/1000
 - 0s - loss: 0.6614 - acc: 0.5880 - val_loss: 0.8031 - val_acc: 0.2167
Epoch 8/1000
 - 0s - loss: 0.6487 - acc: 0.6065 - val_loss: 0.8034 - val_acc: 0.3833
Epoch 9/1000
 - 0s - loss: 0.6156 - acc: 0.6389 - val_loss: 0.7176 - val_acc: 0.6333
Epoch 10/1000
 - 0s - loss: 0.5879 - acc: 0.6991 - val_loss: 0.6562 - val_acc: 0.7167
Epoch 11/1000
 - 0s - loss: 0.5315 - acc: 0.7407 - val_loss: 0.6471 - val_acc: 0.7417
Epoc

In [71]:
scores = model.evaluate(x_test_pca, y_testOneHot)
scores[1]

300/300 [==============================] - 0s 87us/step


0.7400000007947286

In [72]:
prediction = model.predict_classes(x_test_pca)
pd.crosstab(y_test.iloc[:,2],prediction,
            rownames=['Label'],colnames=['Predict'])

Predict,0,1
Label,,
0,207,36
1,42,15


In [73]:
#### confusion table
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

names = ["Healthy eyes", "DR Disease"]
print(classification_report(y_test.iloc[:,2], prediction, target_names = names))


              precision    recall  f1-score   support

Healthy eyes       0.83      0.85      0.84       243
  DR Disease       0.29      0.26      0.28        57

    accuracy                           0.74       300
   macro avg       0.56      0.56      0.56       300
weighted avg       0.73      0.74      0.73       300

